lag変数の追加

In [1]:
from google.colab import drive
drive.mount('/content/drive', force_remount=False)
%cd /content/drive/MyDrive/NEXCO/src/run_test/

Mounted at /content/drive
/content/drive/MyDrive/NEXCO/src/run_test


In [2]:
!pip install japanize_matplotlib jpholiday
# !python run.py  --exec-path ./sample_submit/src/ --data-dir ./train/

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.1/4.1 MB 35.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for japanize_matplotlib: filename=japanize_matplotlib-1.1.3-py3-none-any.whl size=4120274 sha256=634a178b45ac349d27a801408fa4fc9d5294dbc030e75b1009077ab13eb112e0
  Stored in directory: /root/.cache/pip/wheels/91/8f/c2/83055ad0c9591b0a094730aa7cb2cc12fedacbcd2241baf534
Successfully built japanize_matplotlib


In [3]:
"""
train_all = pd.DataFrame()
for d in tqdm(train['datetime'].dt.date.unique()):
    input = train[train['datetime'].dt.date==d]
    input = ScoringService.extract_dataset(input)
    train_all = pd.concat([train_all, input])

search_sec
    "過去7日間の中で", この時間にsection内を通る検索をした数
    >> この日の合計
search_unsec
    "昨日", このsectionを通る検索をした数


# search_secの合計, その日のall 検索hit section数
# search_sec 移動平均
# search_sec それより前or後の検索数 + 上り下り反対車線の
    行だけ or 帰り だけの検索
# *** 周りのsectionとの関連
# 正しく休日を捉える
    連休の中何日めなのか
# {館山, 関越} * {上り, 下り} → 4

# validationの回し方的にlag変数に限界がある
# 2つの車線でmodel分ける??
    館山道: 渋滞少ない 0.13%
    関越道: 渋滞多い 0.53%
# サグ部 下り坂から上り坂


原因: 減速の連鎖
サグや長い上り坂(高坂SAや東松山IC、嵐山PA、前橋IC)
勾配とスピードの変化にドライバーが気付きにくく
スピードが落ちると車間が詰まる
後続車はブレーキを踏んで速度を落とします

"""

'\ntrain_all = pd.DataFrame()\nfor d in tqdm(train[\'datetime\'].dt.date.unique()):\n    input = train[train[\'datetime\'].dt.date==d]\n    input = ScoringService.extract_dataset(input)\n    train_all = pd.concat([train_all, input])\n\nsearch_sec\n    "過去7日間の中で", この時間にsection内を通る検索をした数\n    >> この日の合計\nsearch_unsec\n    "昨日", このsectionを通る検索をした数\n\n\n# search_secの合計, その日のall 検索hit section数\n# search_sec 移動平均\n# search_sec それより前or後の検索数 + 上り下り反対車線の\n    行だけ or 帰り だけの検索\n# *** 周りのsectionとの関連\n# 正しく休日を捉える\n    連休の中何日めなのか\n# {館山, 関越} * {上り, 下り} → 4\n\n# validationの回し方的にlag変数に限界がある\n# 2つの車線でmodel分ける??\n    館山道: 渋滞少ない 0.13%\n    関越道: 渋滞多い 0.53%\n# サグ部 下り坂から上り坂\n\n\n原因: 減速の連鎖\nサグや長い上り坂(高坂SAや東松山IC、嵐山PA、前橋IC)\n勾配とスピードの変化にドライバーが気付きにくく\nスピードが落ちると車間が詰まる\n後続車はブレーキを踏んで速度を落とします\n\n'

In [4]:
# ========================================
# Library
# ========================================
import os
import math
import random
import pickle
import itertools
import warnings
warnings.filterwarnings('ignore')
import sys
import argparse

import numpy as np
import pandas as pd
import datetime
pd.set_option('display.max_columns', None)
import matplotlib.pyplot as plt
import seaborn as sns
import japanize_matplotlib
import jpholiday
from glob import glob
from tqdm import tqdm
from sklearn.model_selection import (
    TimeSeriesSplit,
    StratifiedKFold,
    KFold,
    GroupKFold,
    StratifiedGroupKFold
)# StratifiedGroupKFold, 
from sklearn.utils.class_weight import compute_sample_weight
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import f1_score, roc_auc_score, classification_report
from scipy.optimize import minimize
import lightgbm as lgb

pd.set_option('display.max_columns', None)

def set_seed(seed=0):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)
    # torch.manual_seed(seed)
    # torch.cuda.manual_seed(seed)
    # torch.backends.cudnn.deterministic = True
set_seed()

class TestHoliday(jpholiday.OriginalHoliday):
    def _is_holiday(self, date):
        extra_holidays = [
            #GW
            datetime.date(2021, 4, 29), datetime.date(2022, 4, 29), datetime.date(2023, 4, 29),
            datetime.date(2021, 4, 30), datetime.date(2022, 4, 30), datetime.date(2023, 4, 30),
            datetime.date(2021, 5, 1), datetime.date(2022, 5, 1), datetime.date(2023, 5, 1),
            datetime.date(2021, 5, 2), datetime.date(2022, 5, 2), datetime.date(2023, 5, 2),
            datetime.date(2021, 5, 3), datetime.date(2022, 5, 3), datetime.date(2023, 5, 3),
            datetime.date(2021, 5, 4), datetime.date(2022, 5, 4), datetime.date(2023, 5, 4),
            datetime.date(2021, 5, 5), datetime.date(2022, 5, 5), datetime.date(2023, 5, 5),

            # お盆
            datetime.date(2021, 8, 13), datetime.date(2022, 8, 13), datetime.date(2023, 8, 13),
            datetime.date(2021, 8, 14), datetime.date(2022, 8, 14), datetime.date(2023, 8, 14),
            datetime.date(2021, 8, 15), datetime.date(2022, 8, 15), datetime.date(2023, 8, 15),

            #年末年始
            datetime.date(2021, 12, 31), datetime.date(2022, 12, 31), datetime.date(2023, 12, 31),
            datetime.date(2021, 1, 1), datetime.date(2022, 1, 1), datetime.date(2023, 1, 1),
            datetime.date(2021, 1, 2), datetime.date(2022, 1, 2), datetime.date(2023, 1, 2),
            datetime.date(2021, 1, 3), datetime.date(2022, 1, 3), datetime.date(2023, 1, 3),
        ]
        if date in extra_holidays:
            return True
        return False

# この中を変える
class ScoringService(object):

    @classmethod
    def get_model(cls, model_path, inference_df, TRAIN_MODE=False):
        """Get model method

        Args:
            model_path (str): Path to the trained model directory.
            inference_df: Past data not subject to prediction.

        Returns:
            bool: The return value. True for success.
        """
        cls.data = inference_df
        if len(cls.data) > 0:
            cls.data = cls.extract_row(cls.data)
            cls.data['is_inf_df'] = 1

        cls.cat_cols = ['road_code', 'start_code', 'end_code', 'section', 'direction', 'dayofweek', \
                        'is_holiday', 'is_holiday_1', 'is_holiday_2', 'is_holiday_3', 'is_holiday_-1', 'is_holiday_-2', 'is_holiday_-3', \
                        'holiday_pre_temperature', 'holiday_temperature', 'start_pref_code', 'end_pref_code', 'season']#  , 'max_holiday_temperature', 'sum_holiday_temperature']
        cls.num_cols = ['year', 'month', 'day', 'year_td', 'month_td', 'day_td', 'hour', 'search_1h', 'search_unspec_1d', 'KP', 'start_KP', 'end_KP', 'limit_speed', \
                    'straight_dist', 'KP_se_dist', 'KP_te_dist', 'KP_st_dist', 'KP_max_dist', 'detour_rate', 'start_degree', 'end_degree', 'degree_diss_es', 'OCC', 'allCars', \

                    'search_1h_lag1', 'search_1h_lag2', 'search_1h_lag3', 'search_1h_lag-1', 'search_1h_lag-2', 'search_1h_lag-3', 
                    'search_1h_3mean', 
                    
                    'sum_search_1h', 'mean_search_1h', 'med_search_1h', 'rate_search_1h']

        cls.feature_cols = cls.cat_cols + cls.num_cols

        cls.TRAIN_MODE = TRAIN_MODE

        cls.N_SPLIT = 5
        if not TRAIN_MODE:
            with open('../src/features/le_dict.pkl', 'rb') as f:
                cls.le = pickle.load(f)
            print(cls.le)
            cls.le_keys = list(cls.le.keys())
            # modelのロード
            cls.models = {}
            for n in range(cls.N_SPLIT):
                cls.models[n] = pickle.load(open(f'{model_path}/lgb_fold{n}.pickle', 'rb'))
        else:
            cls.models = None

        cls.params = {
            'objective': 'binary',
            'verbose': -1,
            'metric': 'binary_logloss',
            'seed': 0,

            'num_leaves': 52, 
            'learning_rate': 0.01, 
            'max_depth': 9,
            'min_data_in_leaf': 10, 
            'feature_fraction': 0.9351779629995216, 
            'bagging_fraction': 0.6384393631575852, 
            'lambda_l1': 0.9764594650133958, 
            'lambda_l2': 0.4686512016477016
        }

        # 一応過去のレコードを貯めることはできる
        return True

    @classmethod
    def expand_datetime(cls, df):
        if 'datetime' in df.columns:
            df['year'] = df['datetime'].dt.year
            df['month'] = df['datetime'].dt.month
            df['day'] = df['datetime'].dt.day
            df['hour'] = df['datetime'].dt.hour
        if 'td' in df.columns:
            df['year_td'] = df['td'].dt.year
            df['month_td'] = df['td'].dt.month
            df['day_td'] = df['td'].dt.day
            df['hour_td'] = df['td'].dt.hour
        if 'date' in df.columns:
            df['year'] = df['date'].dt.year
            df['month'] = df['date'].dt.month
            df['day'] = df['date'].dt.day
        return df

    @classmethod
    def extract_row(cls, df):
        '''
        1行内で完結する特徴量抽出
        inference_dfにもinputにも共通して働く
        流れとしては
            inference_df > {ext_row} > inference_df'
            input > {ext_row} > input'
        で条件を揃える
        '''
        def is_jpholiday(date):
            if jpholiday.is_holiday(date) or 5<=date.weekday():
                return True
            else:
                return False
        # ========================================
        # 時間関連
        # 予測当日の日付
        # ========================================
        df['td'] = df['datetime'] + pd.to_timedelta(1, 'd')

        df['dayofweek'] = df['datetime'].dt.weekday
        df['dayofweek_td'] = df['td'].dt.weekday

        date = df.iloc[0]['td'].date()
        df['is_holiday'] = int(is_jpholiday(date))

        holi_lag = 3# 前後3日分を考慮
        for i in range(holi_lag):# 0,1,2
            lag = i+1# 1,2,3
            df[f'is_holiday_{lag}'] = int(is_jpholiday(date+pd.to_timedelta(lag, 'd')))
            df[f'is_holiday_{-lag}'] = int(is_jpholiday(date+pd.to_timedelta(-lag, 'd')))

        df['holiday_temperature'] = df['is_holiday_1']*4 + df['is_holiday_2']*2 + df['is_holiday_3']*1
        df['holiday_pre_temperature'] = df['is_holiday_-1']*4 + df['is_holiday_-2']*2 + df['is_holiday_-3']*1

        #df['max_holiday_temperature'] = df[['holiday_temperature', 'holiday_pre_temperature']].max(axis=1)
        #df['sum_holiday_temperature'] = df[['holiday_temperature', 'holiday_pre_temperature']].sum(axis=1)

        # date = df.iloc[0]['td'].date()
        # df['is_holiday_td'] = int(is_jpholiday(date))

        df = cls.expand_datetime(df)
        df['season'] = df['month_td'].apply(lambda x: (x % 12 + 3) // 3)

        # ========================================
        # 道路関連
        # ========================================
        df['section'] = df['start_code'].astype(str)+'_'+df['end_code'].astype(str)
        df['straight_dist'] = np.sqrt( (df['end_lat']-df['start_lat'])**2 + (df['end_lng']-df['start_lng'])**2 ) * 100
        df['KP_se_dist'] = np.abs(df['end_KP']-df['start_KP'])
        df['KP_te_dist'] = np.abs(df['end_KP']-df['KP'])
        df['KP_st_dist'] = np.abs(df['KP']-df['start_KP'])
        df['KP_max_dist'] = np.max(df[['KP_te_dist', 'KP_st_dist']], axis=1) / df['KP_se_dist']
        df['detour_rate'] = df['KP_se_dist']/df['straight_dist']
        df['degree_diss_es'] = df['end_degree'] - df['start_degree']

        return df

    @classmethod
    def extract_with_inference(cls, df):
        '''
        df = 推論対象1日分のデータ
        cls.data = 過去データ

        input'' = concat(input', inference_df').reset_index()
        sort(input'', by={'datetime'})
            input''.groupby('section')# section毎の操作, lag変数等
            ~~ # 他のsectionとの操作 
        '''
        df['is_inf_df'] = 0
        df = pd.concat([cls.data, df]).sort_values('datetime').reset_index()


        # ========================================
        # lag変数 group毎集計
        # ========================================
        df_g = df.groupby('section')
        lags = [1,2,3,-1,-2,-3]# 3時間前から3時間後まで
        for lag in lags:
            df[f'search_1h_lag{lag}'] = df_g.search_1h.shift(lag).reset_index().set_index('index')['search_1h']

        df['search_1h_3mean'] = df_g.search_1h.rolling(3, min_periods=1).mean().reset_index().set_index('level_1')['search_1h']
        df = df.replace([np.nan], -1)
        df = df.replace([np.inf], 999999)
        df = df.replace([-np.inf], -999999)

        # ========================================
        # drop inference_df + 登録
        # ========================================
        df = df[df['is_inf_df']==0]
        df['is_inf_df'] = 1
        df = df.set_index('index').sort_index()
        cls.data = df # pd.concat([cls.data, df])


        # ========================================
        # ルート検索 関連 inference関係なし
        # ========================================
        group_search = df.groupby('section').agg(
            mean_search_1h = ("search_1h", 'mean'),
            sum_search_1h = ("search_1h", 'sum'),
            med_search_1h = ("search_1h", 'median')
            ).reset_index()
        df = pd.merge(df.reset_index(), group_search, on='section', suffixes=('', '_grouped'), how='left').set_index('index')
        df['rate_search_1h'] = df['search_1h'] / df['sum_search_1h']
        df['rate_search_1h'] = df['rate_search_1h'].replace([np.inf, -np.inf, np.nan], -1)

        return df


    @classmethod
    def extract_dataset(cls, df):
        '''
        input: df
            df = valid[valid['datetime'].dt.date==d]
            1日分のデータフレーム(1d*24h*79section)
            予測日前日のtraffic + 予測日のsearch + road >> 予測

        output: ~~
        '''

        df = cls.extract_row(df)# 1日分
        df = cls.extract_with_inference(df)

        # ========================================
        # カテゴリ変数の処理
        # ========================================
        if not cls.TRAIN_MODE:
            for c in cls.le_keys:
                df[c] = cls.le[c].transform(df[c])

        return df

    @classmethod
    def predict(cls, input):
        """Predict method

        Args:
            input: meta data of the sample you want to make inference from (DataFrame)

        Returns:
            prediction: Inference for the given input. Return columns must be ['datetime', 'start_code', 'end_code'](DataFrame).
        
        Tips:
            You can use past data by writing "cls.data".
        """
        
        prediction = input.copy()

        # ======================== ここから特徴量作成 ========================

        prediction = cls.extract_dataset(prediction)

        # ======================== ここから予測 =============================

        prediction['prediction'] = 0
        for n in range(cls.N_SPLIT):
            prediction['prediction'] += cls.models[n].predict(prediction[cls.feature_cols]) / cls.N_SPLIT
        prediction['prediction'] = prediction['prediction'].round()
        prediction['prediction'] = prediction['prediction'].astype(int)
        prediction['start_code'] = input["start_code"]
        prediction['end_code'] = input["end_code"]
        prediction['datetime'] = input["datetime"]
        # prediction = prediction[['datetime', 'start_code', 'end_code', 'prediction']]

        return prediction

# -----

In [5]:

# 不変
def make_dataset(traffic, ic_master, search_spec, search_unspec):
    # 欠損値の除外
    traffic = traffic[traffic['speed'].isnull()==False]
    ic_master.dropna(inplace=True)
    search_spec.dropna(inplace=True)
    search_unspec.dropna(inplace=True)
    
    # datetimeからdateを作成
    traffic['date'] = traffic['datetime'].apply(lambda x: x.split()[0])

    # データのマージ
    traffic = traffic.merge(ic_master, on=['start_code', 'end_code'], how='left')
    traffic = traffic.merge(search_spec, on=['datetime', 'start_code', 'end_code'], how='left')
    traffic = traffic.merge(search_unspec, on=['date', 'start_code', 'end_code'], how='left')
    traffic.sort_values(['date', 'start_code', 'end_code'], inplace=True)
    traffic.reset_index(drop=True, inplace=True)
    traffic.drop(columns='date', inplace=True)

    # データ型の変更
    traffic['datetime'] = pd.to_datetime(traffic['datetime'])

    return traffic


def parse_args():
    parser = argparse.ArgumentParser()
    parser.add_argument('--exec-path', help = '/path/to/submit/src')
    parser.add_argument('--data-dir', help = '/path/to/train')
    parser.add_argument('--start-date', default = '2021-04-08', type = str, help='start date')
    parser.add_argument('--end-date', default = '2022-07-31', type = str, help='end date')
    args = parser.parse_args()

    return args

def train():

    # ========================================
    # training module
    # ========================================
    def f1(y_true, y_pred):
        y_pred = (y_pred > 0.5).astype(int)
        score = f1_score(y_true, y_pred)
        return 'f1', score, True

    def lgb_f1_score(y_hat, data):
        y_true = data.get_label()
        y_hat = np.where(y_hat < 0.5, 0, 1)   # scikits f1 doesn't like probabilities
        return 'f1', f1_score(y_true, y_hat), True

    def train_lgbm(X,
                y,
                cv,
                model_path = [],
                params: dict=None,
                verbose: int=100,
                weight_dict: dict=None,
                cat_cols: list=[]
                ):

        # パラメータがないときは、空の dict で置き換える
        if params is None:
            params = {}

        models = []

        n_records = len(X)
        # training data の target と同じだけのゼロ配列を用意
        oof_pred = np.zeros((n_records, ), dtype=np.float32)
        
        for i, (idx_train, idx_valid) in enumerate(cv):
            x_train, y_train = X.iloc[idx_train], y.iloc[idx_train]
            x_valid, y_valid = X.iloc[idx_valid], y.iloc[idx_valid]

            print(f'train shape = {x_train.shape}')
            print(f'valid shape = {x_valid.shape}')
            
            if weight_dict is None:
                print('Don\'t use sample weight.')
                lgb_train = lgb.Dataset(x_train, y_train)
            elif weight_dict[0] == 'balance':
                print('Using sample weight. {Balance}')
                lgb_train = lgb.Dataset(x_train, y_train, weight=compute_sample_weight(class_weight='balanced', y=y_train).astype('float32'))
            else:
                print('Using sample weight.')
                print(weight_dict)
                lgb_train = lgb.Dataset(x_train, y_train, weight=compute_sample_weight(class_weight=weight_dict, y=y_train).astype('float32'))
                
            lgb_eval = lgb.Dataset(x_valid, y_valid, reference=lgb_train)
            
            clf = lgb.train(params, lgb_train, 
                            valid_sets=[lgb_train, lgb_eval],
                            valid_names=['Train', 'Valid'],
                            feval=lgb_f1_score,
                            num_boost_round=100000,
                            verbose_eval=100,
                            categorical_feature=cat_cols,
                            callbacks = [lgb.early_stopping(stopping_rounds=100, verbose=True)]
                            )# categorical_feature=categories, verbose_eval=10
            
            pred_i = clf.predict(x_valid)
            oof_pred[idx_valid] = pred_i
            models.append(clf)
            score = f1_score(y_valid, (pred_i > 0.5).astype(int))
            print(f" - fold{i + 1} - {score:.4f} \n")
            
        score = f1_score(y, (oof_pred > 0.5).astype(int))

        print("=" * 50)
        print(f"FINISH: CV Score: {score:.4f}")
        return score, oof_pred, models

    # parse the arguments
    """
    args = parse_args()
    exec_path = os.path.abspath(args.exec_path)
    data_dir = os.path.abspath(args.data_dir)
    start_date = args.start_date
    end_date = args.end_date

    !python run.py  --exec-path ./sample_submit/src/ --data-dir ./train/
    """
    exec_path = '/content/drive/MyDrive/NEXCO/src/run_test/sample_submit/src'
    data_dir = '/content/drive/MyDrive/NEXCO/src/run_test/train/'
    start_date = '2021-04-08'
    end_date = '2022-07-31'

    print('\nstart date: {}, end date:{}'.format(start_date, end_date))

    # load the input data
    print('\nLoading Dataset...')
    traffic = pd.read_csv(os.path.join(data_dir, 'train.csv'))
    search_spec = pd.read_csv(os.path.join(data_dir, 'search_data.csv'))
    search_unspec = pd.read_csv(os.path.join(data_dir, 'search_unspec_data.csv'))
    ic_master = pd.read_csv(os.path.join(data_dir, 'road.csv'))

    # 当日の検索数を使用できるように変更(search_spec, search_unspec)
    search_spec['datetime'] = pd.to_datetime(search_spec['datetime'])
    search_unspec['date'] = pd.to_datetime(search_unspec['date'])
    search_spec['datetime'] -= pd.to_timedelta(1, 'd')
    search_unspec['date'] -= pd.to_timedelta(1, 'd')
    search_spec['datetime'] = search_spec['datetime'].astype('str')
    search_unspec['date'] = search_unspec['date'].astype('str')

    df = make_dataset(traffic, ic_master, search_spec, search_unspec)
    inference_df = df[df['datetime']<start_date+' 00:00:00']
    train = df[(df['datetime']>=start_date+' 00:00:00') & (df['datetime']<=end_date+' 23:00:00')]
    print('Done')

    # change the working directory
    model_path = './sample_submit/model/'
    ScoringService.get_model(model_path, inference_df, TRAIN_MODE=True)

    # prepare training data
    train_all = pd.DataFrame()
    for d in tqdm(train['datetime'].dt.date.unique()):
        input = train[train['datetime'].dt.date==d]
        input = ScoringService.extract_dataset(input)
        train_all = pd.concat([train_all, input])

    le_dict = {}# only training
    for c in ScoringService.cat_cols:
        if train_all[c].dtype=='object':
            le = LabelEncoder()# only training
            print(c)
            print(train_all[c].value_counts())
            le.fit(train_all[c])# only training
            train_all[c] = le.transform(train_all[c])
            le_dict[c] = le# only training

    with open("./sample_submit/src/features/le_dict.pkl", "wb") as f:# only training
        pickle.dump(le_dict, f)# only training

    # training
    train_all.sort_values(['section', 'datetime'])
    train_all['is_congestion'] = train_all.groupby('section').shift(-24).reset_index()['is_congestion']
    train_all = train_all[~train_all['is_congestion'].isnull()].reset_index(drop=True)

    # ========================================
    # train-validation split
    # ========================================
    TARGET = 'is_congestion'
    N_SPLIT = ScoringService.N_SPLIT
    kf = StratifiedGroupKFold(N_SPLIT)
    cv_list = list(kf.split(train_all, y=train_all[TARGET], groups=train_all['datetime'].dt.date))

    X = train_all[ScoringService.feature_cols]
    y = train_all[TARGET]
    print('train all shape:', X.shape)
    print('train columns: \n', X.columns)

    weight = {0:1.0, 1:5.0}
    score, oof_pred, models = train_lgbm(X, y=y, params=ScoringService.params, cv=cv_list, weight_dict=weight, cat_cols=ScoringService.cat_cols)# 0.5814 <> shuffle 5810

    for i, m in enumerate(models):
        with open(f'./sample_submit/model/lgb_fold{i}.pickle', mode="wb") as f:
            pickle.dump(m, f)

    # ========================================
    # feature importance
    # ========================================
    def visualize_importance(models, feat_train_df):
        feature_importance_df = pd.DataFrame()
        for i, model in enumerate(models):
            _df = pd.DataFrame()
            importance_type = 'split'
            _df["feature_importance"] = model.feature_importance(importance_type=importance_type)
            _df["column"] = feat_train_df.columns
            _df["fold"] = i + 1
            feature_importance_df = pd.concat([feature_importance_df, _df],
                                            axis=0, ignore_index=True)

        order = feature_importance_df.groupby("column")\
            .sum()[["feature_importance"]]\
            .sort_values("feature_importance", ascending=False).index

        fig, ax = plt.subplots(figsize=(12, max(6, len(order) * .25)))
        sns.boxplot(data=feature_importance_df,
                    x="feature_importance",
                    y="column",
                    order=order,
                    ax=ax,
                    palette="viridis",
                    orient="h")
        ax.tick_params(axis="x", rotation=90)
        ax.set_title("Importance")
        ax.grid()
        fig.tight_layout()
        return fig, ax, feature_importance_df

    fig, ax, feature_importance_df = visualize_importance(models, X)

def main():

    # parse the arguments
    """
    args = parse_args()
    exec_path = os.path.abspath(args.exec_path)
    data_dir = os.path.abspath(args.data_dir)
    start_date = args.start_date
    end_date = args.end_date

    !python run.py  --exec-path ./sample_submit/src/ --data-dir ./train/
    """
    exec_path = '/content/drive/MyDrive/NEXCO/src/run_test/sample_submit/src'
    data_dir = '/content/drive/MyDrive/NEXCO/src/run_test/train/'
    start_date = '2021-04-08'
    end_date = '2022-07-31'

    print('\nstart date: {}, end date:{}'.format(start_date, end_date))

    # load the input data
    print('\nLoading Dataset...')
    traffic = pd.read_csv(os.path.join(data_dir, 'train.csv'))
    search_spec = pd.read_csv(os.path.join(data_dir, 'search_data.csv'))
    search_unspec = pd.read_csv(os.path.join(data_dir, 'search_unspec_data.csv'))
    ic_master = pd.read_csv(os.path.join(data_dir, 'road.csv'))

    # 当日の検索数を使用できるように変更(search_spec, search_unspec)
    search_spec['datetime'] = pd.to_datetime(search_spec['datetime'])
    search_unspec['date'] = pd.to_datetime(search_unspec['date'])
    search_spec['datetime'] -= pd.to_timedelta(1, 'd')
    search_unspec['date'] -= pd.to_timedelta(1, 'd')
    search_spec['datetime'] = search_spec['datetime'].astype('str')
    search_unspec['date'] = search_unspec['date'].astype('str')

    df = make_dataset(traffic, ic_master, search_spec, search_unspec)
    train = df[df['datetime']<start_date+' 00:00:00']
    valid = df[(df['datetime']>=start_date+' 00:00:00') & (df['datetime']<=end_date+' 23:00:00')]
    print('Done')

    # change the working directory
    os.chdir(exec_path)
    cwd = os.getcwd()
    print('\nMoved to {}'.format(cwd))
    model_path = os.path.join('..', 'model')
    print('Model papth = {}'.format(model_path))
    sys.path.append(cwd)

    # load the model
    # from predictor import ScoringService

    print('\nLoading the model...', end = '\r')
    model_flag = ScoringService.get_model(model_path, train)
    if model_flag:
        print('Loaded the model.')
    else:
        print('Could not load the model.')
        return None

    predictions = pd.DataFrame()
    for d in tqdm(valid['datetime'].dt.date.unique()):
        input = valid[valid['datetime'].dt.date==d]
        prediction = ScoringService.predict(input)
        if type(prediction)!= pd.DataFrame:
            print('Invalid data type in the prediction. Must be pandas.DataFrame')
            return None 
        '''elif set(prediction.columns) != set(['datetime', 'start_code', 'end_code', 'prediction']):
            print('Invalid columns name: {},  Excepted name: {}'.format(prediction.columns, {'datetime', 'start_code', 'end_code', 'prediction'}))
            return None'''
        predictions = pd.concat([predictions, prediction])

    results = valid[['datetime', 'start_code', 'end_code']]
    results = pd.merge(results, predictions[['datetime', 'start_code', 'end_code', 'prediction']], on=['datetime', 'start_code', 'end_code'], how='left')
    results['datetime'] = pd.to_datetime(results['datetime'])
    results['datetime'] += pd.to_timedelta(1, 'd')
    results = pd.merge(results, df[['datetime', 'start_code', 'end_code', 'is_congestion']], on=['datetime', 'start_code', 'end_code'], how='inner')

    #compute F1SCORE
    print('\n==================')
    print('f1_score:', round(f1_score(results['is_congestion'], results['prediction']), 4))
    print('==================')

    return train, valid, predictions


 # train & main

In [ ]:
train()


start date: 2021-04-08, end date:2022-07-31

Loading Dataset...
Done


100%|██████████| 480/480 [05:06<00:00,  1.57it/s]


section
1110210_1800006    11520
1800086_1800081    11520
1800106_1800111    11520
1800106_1800096    11520
1800096_1800106    11520
                   ...  
1800026_1800021    11520
1800021_1800026    11520
1800021_1800016    11520
1800016_1800021    11520
1800183_1800176    11520
Name: section, Length: 79, dtype: int64
direction
上り    460800
下り    449280
Name: direction, dtype: int64
train all shape: (908184, 53)
train columns: 
 Index(['road_code', 'start_code', 'end_code', 'section', 'direction',
       'dayofweek', 'is_holiday', 'is_holiday_1', 'is_holiday_2',
       'is_holiday_3', 'is_holiday_-1', 'is_holiday_-2', 'is_holiday_-3',
       'holiday_pre_temperature', 'holiday_temperature', 'start_pref_code',
       'end_pref_code', 'season', 'year', 'month', 'day', 'year_td',
       'month_td', 'day_td', 'hour', 'search_1h', 'search_unspec_1d', 'KP',
       'start_KP', 'end_KP', 'limit_speed', 'straight_dist', 'KP_se_dist',
       'KP_te_dist', 'KP_st_dist', 'KP_max_dist', 'detour_

In [ ]:
train, valid, predictions = main()

# EDA

In [ ]:
valid.head(30)

In [ ]:
predictions.head(30)

In [ ]:
valid.tail(30)

In [ ]:
predictions.tail(30)

In [ ]:
def is_jpholiday(date):
    if jpholiday.is_holiday(date) or 5<=date.weekday():
        return True
    else:
        return False

# Validationの方
val_year = '2022-'
val_days = ['08-06', '08-07', 
            '08-10', '08-11', '08-12', '08-13', '08-14', '08-15', '08-16', 
            '08-20', '08-21', 
            '08-27', '08-28', 
            '09-03', '09-04', 
            '09-10', '09-11', 
            '09-17', '09-18', '09-19', '09-20', '09-21', '09-22', '09-23', '09-24', '09-25']

tes_year = '2023-'
tes_days = ['04-01', '04-02', 
            '04-08', '04-09',
            '04-15', '04-16',
            '04-22', '04-23',
            '04-29', '04-30', '05-01', '05-02', '05-03', '05-04', '05-05', '05-06', '05-07']

# 指定の日って 'is_holiday_temperature' いくつ??
temperature_list = []
pre_temperature_list = []
max_temperature_list = []
sum_temperature_list = []

for vd in val_days:
    today = pd.to_datetime(val_year+vd)
    print('\n================= valid ===================')
    print('today = ', today, 'is_holiday? >> ', jpholiday.is_holiday(today))

    holi_lag = 3# 前後3日分を考慮

    is_holiday_temperature = 0
    is_holiday_pre_temperature = 0
    for i in range(holi_lag):# 0,1,2
        lag = i+1
        is_holi = int(is_jpholiday(today+pd.to_timedelta(lag, 'd')))# 0or1
        is_holi_pre = int(is_jpholiday(today+pd.to_timedelta(-lag, 'd')))# 0or1

        if i==0:
            is_holiday_temperature += is_holi*4
            is_holiday_pre_temperature += is_holi_pre*4
        elif i==1:
            is_holiday_temperature += is_holi*2
            is_holiday_pre_temperature += is_holi_pre*2
        else:
            is_holiday_temperature += is_holi*1
            is_holiday_pre_temperature += is_holi_pre*1

    temperature_list.append(is_holiday_temperature)
    pre_temperature_list.append(is_holiday_pre_temperature)
    max_temperature_list.append(max(is_holiday_temperature, is_holiday_pre_temperature))
    sum_temperature_list.append(is_holiday_temperature+is_holiday_pre_temperature)

for td in tes_days:
    today = pd.to_datetime(tes_year+td)
    print('\n================== test ==================')
    print('today = ', today, 'is_holiday? >> ', jpholiday.is_holiday(today))

    holi_lag = 3# 前後3日分を考慮

    is_holiday_temperature = 0
    is_holiday_pre_temperature = 0
    for i in range(holi_lag):# 0,1,2
        lag = i+1
        is_holi = int(is_jpholiday(today+pd.to_timedelta(lag, 'd')))# 0or1
        is_holi_pre = int(is_jpholiday(today+pd.to_timedelta(-lag, 'd')))# 0or1

        if i==0:
            is_holiday_temperature += is_holi*4
            is_holiday_pre_temperature += is_holi_pre*4
        elif i==1:
            is_holiday_temperature += is_holi*2
            is_holiday_pre_temperature += is_holi_pre*2
        else:
            is_holiday_temperature += is_holi*1
            is_holiday_pre_temperature += is_holi_pre*1

    temperature_list.append(is_holiday_temperature)
    pre_temperature_list.append(is_holiday_pre_temperature)
    max_temperature_list.append(max(is_holiday_temperature, is_holiday_pre_temperature))
    sum_temperature_list.append(is_holiday_temperature+is_holiday_pre_temperature)
    
print('')
print(f'[temperature] max: {max(temperature_list)}  min: {min(temperature_list)}')
print(f'[pre_temperature] max: {max(pre_temperature_list)}  min: {min(pre_temperature_list)}')
print(f'[max_temperature] max: {max(max_temperature_list)}  min: {min(max_temperature_list)}')
print(f'[max_temperature] max: {max(sum_temperature_list)}  min: {min(sum_temperature_list)}')

In [ ]:
max(predictions['holiday_temperature']), min(predictions['holiday_temperature'])

In [ ]:
max(predictions['holiday_pre_temperature']), min(predictions['holiday_pre_temperature'])

In [ ]:
max(predictions['max_holiday_temperature']), min(predictions['max_holiday_temperature'])

In [ ]:
max(predictions['sum_holiday_temperature']), min(predictions['sum_holiday_temperature'])

In [ ]:
predictions['max_holiday_temperature'].value_counts().sort_index()

In [ ]:
predictions['sum_holiday_temperature'].value_counts().sort_index()

In [ ]:
predictions.groupby('holiday_temperature').is_congestion.mean().reset_index()

In [ ]:
predictions.groupby('holiday_pre_temperature').is_congestion.mean().reset_index()

In [ ]:
predictions.groupby('max_holiday_temperature').is_congestion.mean().reset_index()

In [ ]:
predictions.groupby('sum_holiday_temperature').is_congestion.mean().reset_index()

In [ ]:
shift_flag = 0

In [ ]:
# 月と目的変数の関係
if shift_flag:
    predictions.sort_values(['section', 'datetime'])
    predictions['is_congestion'] = predictions.groupby('section').shift(-24).reset_index()['is_congestion']
    predictions = predictions[~predictions['is_congestion'].isnull()].reset_index(drop=True)
    shift_flag=0

mean_target_grouped_month = predictions.groupby('month').is_congestion.mean().reset_index()
plt.bar(mean_target_grouped_month['month'], mean_target_grouped_month['is_congestion'])